In [1]:
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.8 MB/s eta 0:00:00


In [2]:
!pip install -q langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.8/166.8 kB 1.7 MB/s eta 0:00:00


In [3]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

In [4]:
base_url = "https://7d96-104-199-237-94.ngrok-free.app"
model = OllamaFunctions(model="llama2",base_url = base_url)

In [5]:
model = model.bind(
    functions=[
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, " "e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)

In [6]:
from langchain.schema import HumanMessage

model.invoke("what is the weather in Boston?")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "Boston", "unit": "celsius"}'}})

In [7]:
from langchain.chains import create_extraction_chain

# Schema
schema = {
    "properties": {
        "name": {"type": "string"},
        "height": {"type": "integer"},
        "hair_color": {"type": "string"},
    },
    "required": ["name", "height"],
}

# Input
input = """Alex is 5 feet tall. Claudia is 1 feet taller than Alex and jumps higher than him. Claudia is a brunette and Alex is blonde."""

# Run chain
chain = create_extraction_chain(schema, model)
chain.run(input)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[{'name': 'Alex', 'height': 5, 'hair_color': 'blonde'},
 {'name': 'Claudia', 'height': 6, 'hair_color': 'brunette'}]

In [8]:
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "language": {"type": "string"},
    }
}
chain = create_tagging_chain(schema, model)

In [9]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.run(inp)

{'sentiment': {'value': 'positive', 'language': 'es'},
 'aggressiveness': {'value': 0, 'language': 'es'},
 'language': {'value': 'es'}}

In [10]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain.run(inp)

{'sentiment': {'title': 'agressive', 'type': 'string'},
 'aggressiveness': {'title': 'high', 'type': 'integer'},
 'language': {'title': 'Spanish', 'type': 'string'}}

In [11]:
from langchain_core.pydantic_v1 import BaseModel, Field

In [12]:
class Tags(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [13]:
chain = create_tagging_chain_pydantic(Tags, model)

In [14]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
res = chain.run(inp)

In [15]:
res

Tags(sentiment='aggressive', aggressiveness=4, language='spanish')